In [ ]:
# Notes for saving and loading Pytorch Models, from 
# https://pytorch.org/tutorials/beginner/saving_loading_models.html
'''
model = TheModelClass(*args, **kwargs)
optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()
'''
# Useful reference for saving off weights, etc, that we might need and/or want

In [ ]:
class customLoss():
    def __init__(self):
        self.pi = math.pi
        
    def forward(self, y, yhat):
        self.Errs = torch.cat((torch.remainder(y-yhat, 2.0*self.pi), torch.sub(torch.remainder(y-yhat, 2.0*self.pi), 2.0*self.pi)), 1)
        tmp1 = torch.min(torch.abs(self.Errs), 1).values
        return torch.sum(tmp1)
        # return torch.sum(torch.min(torch.abs(torch.tensor([self.Errs, self.Errs-2.0*math.pi])), 1))



In [ ]:
# Define needed learning quantities

# Learning rate (initial)
# ### Generally ~ 1e-3 for Adam
learning_rate = 1e-3

# Batch size (default 64)
batch_size_train = 180
batch_size_test = int(0.4*batch_size_train)


In [ ]:
def train_loop(dataloader, model, optimizer, epochNum, seedNum, loss_fn=nn.L1Loss(), verbose=True, batch_size=180, writePath=None, loocv=None):
    """
    Loop for training a 'model' (class NeuralNetwork) on data stored in 'dataloader,' using loss function
    'loss_fn' and optimizer method 'optimizer'
    
    [Inputs]
    dataloader    -- type DataLoader    -- Pytorch DataLoader object to facilitate training/testing data storage
                                           to interface with pytorch optimization and training modules
                                           
    model         -- type NeuralNetwork -- Pytorch NeuralNetwork object to facilitate training/testing of speed and 
                                           angle prediction for FlowDrone
                                           
    epochNum      -- type int           -- Current epoch number to track training loss
    
    seedNum       -- type int           -- Seed of the current run (to average over to demonstrate convergence)
    
    loss_fn       -- type torch.nn loss -- Pytorch loss function (in nn library) for training the speed/angle predictor
                                           for FlowDrone. Defaults to nn.MSELoss() because we are regressing real-valued
                                           variables. 
                                           
    optimizer     -- type torch.optim   -- Pytorch optimizer for ANN weight updates. Normally will use ADAM unless there
                                           is a compelling reason to deviate. 
    
    verbose       -- type Boolean       -- Toggles printing of training loss during training. Default is TRUE. 
    
    writePath     -- type String        -- If present, a path to write to a csv file in order 
    [Outputs]
    
    None

    """
    size = len(dataloader.dataset)
    f = open(writePath+'trainCost'+str(seedNum)+'.csv', 'a') if loocv is None else open(writePath+'valCost'+str(seedNum)+'_'+str(loocv)+'.csv', 'a')
    writer = csv.writer(f)
    
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        '''
        for k in range(pred.shape[0]):
            if ((pred[k, -1] - y[k, -1]) >= math.pi):
                while ((pred[k, -1] - y[k, -1]) >= math.pi):
                    pred[k, -1] -= 2.0*math.pi
            elif ((pred[k, -1] - y[k, -1]) <= -math.pi):
                pred[k, -1] = (pred[k,-1])%(2.0*math.pi)
        '''
        loss = loss_fn.forward(y, pred)
        loss_average = (loss/pred.shape[0]).cpu().detach().numpy()
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 200 == 199 or (batch==0 and epochNum==1):
            if verbose:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
            if writePath is None:
                pass
            else: 
                writer.writerow(np.array([(batch + (np.ceil(size/batch_size)*epochNum)), 180.0*loss_average/math.pi]))
                

    # close the file
    f.close()
    return 

#

In [ ]:
def test_loop(dataloader, model, epochNum, seedNum, loss_fn=nn.L1Loss(), lastLoop=False, writePath=None, loocv=None):
    """
    Loop for test a 'model' (class NeuralNetwork) on data stored in 'dataloader,' using loss function
    'loss_fn.'
    
    [Inputs]
    dataloader    -- type DataLoader    -- Pytorch DataLoader object to facilitate training/testing data storage
                                           to interface with pytorch optimization and training modules
                                           
    model         -- type NeuralNetwork -- Pytorch NeuralNetwork object to facilitate training/testing of speed and 
                                           angle prediction for FlowDrone

    epochNum      -- type Int           -- Current epoch
    
    loss_fn       -- type torch.nn loss -- Pytorch loss function (in nn library) for training the speed/angle predictor
                                           for FlowDrone. Defaults to nn.MSELoss() because we are regressing real-valued
                                           variables. 
                                           
    lastLoop      -- type Boolean       -- Whether we are on the last epoch (for histogram information)
    
    writePath     -- type String        -- File to write to
    [Outputs]
    
    None

    """
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    batch_size = dataloader.batch_size
    hists = False
    
    if lastLoop:
        if (num_batches == size):
            print('On the last loop!')
            errs = np.zeros((size, 2))
            idxVal = 0
            hists = True
        
    test_loss = 0.0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            '''
            for k in range(pred.shape[0]):
                if ((pred[k, -1] - y[k, -1]) >= math.pi):
                    while ((pred[k, -1] - y[k, -1]) >= math.pi):
                        pred[k, -1] -= 2.0*math.pi
                elif ((pred[k, -1] - y[k, -1]) <= -math.pi):
                    pred[k, -1] = (pred[k,-1])%(2.0*math.pi)
            '''
            if hists:
                errs[idxVal, :] = np.array([y, loss_fn.forward(y, pred)])
                # errs[idxVal, :] = np.array([y, loss_fn(y, pred).item()])
                test_loss += errs[idxVal, 1]
                idxVal += 1
            else:
                test_loss += loss_fn.forward(y, pred)/pred.shape[0]

    test_loss /= num_batches
    if loocv is None:
        print(f"Avg loss (rad): {test_loss:>8f}")
        print(f"Avg error (deg): {(test_loss*180.0/math.pi):>8f} \n")
    else: 
        print(f"Avg loss/error (m/s): {test_loss:>8f}")

    if writePath is None:
        pass
    else:
        # open the file
        f = open(writePath+'testCost'+str(seedNum)+'.csv', 'a') if loocv is None else open(writePath+'valCost'+str(seedNum)+'_'+str(loocv)+'.csv', 'a')
        writer = csv.writer(f)
        
        # write out the relevant cost
        writer.writerow(np.array([epochNum, test_loss*180.0/math.pi]))
                
        # close the file
        f.close()
    
    if hists:
        if loocv is None:
            np.savetxt(writePath+'testErrs'+str(seedNum)+'.csv', errs, delimiter=',')
        else: 
            np.savetxt(writePath+'valErrs'+str(seedNum)+'_'+str(loocv)+'.csv', errs, delimiter=',')
        
        return test_loss, errs
    
    else: 
        return test_loss

#
